## Chapter 9. Decorators and Closures

### Decorators 101
A decorator is a callable that takes another function as an argument (the decorated
function).

In other words, assuming an existing decorator named decorate, this code:
```
@decorate
def target():
    print('running target()')
```
has the same effect as writing this:
```
def target():
    print('running target()')

target = decorate(target)
```

In [1]:
# Example 9-1. A decorator usually replaces a function with a different one

def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [2]:
target()
target  # target is a reference to inner

running inner()


<function __main__.deco.<locals>.inner()>

In [4]:
# Example 9-2. The registration.py module

registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register  # function decorators are executed as soon as the module is imported.
def f1():  # the decorated fuction only runs when it is executed explicitly invoked.
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

print()
print('running main()')
print('registry ->', registry)
f1()
f2()
f3()

running register(<function f1 at 0x00000245A007FA30>)
running register(<function f2 at 0x00000245A007FAC0>)

running main()
registry -> [<function f1 at 0x00000245A007FA30>, <function f2 at 0x00000245A007FAC0>]
running f1()
running f2()
running f3()


### Variable Scope Rules

In [5]:
# Example 9-3. Function reading a local and a global variable

def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [6]:
b = 6
f1(3)

3
6


In [12]:
# Example 9-4. Variable b is local, because it is assigned a value in the body of the function
b = 6

def f2(a):
    print(a)
    print(b)  # error since Python compiles the body of the function, it decides that b is a local variable
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [9]:
b = 6

def f2_g(a):
    global b
    print(a)
    print(b)
    b = 9

f2_g(3)
print(b)

3
6
9


In [10]:
# Example 9-5. Disassembly of the f1 function from Example 9-3

from dis import dis
dis(f1)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [13]:
# Example 9-6. Disassembly of the f2 function from Example 9-4
dis(f2)

  5           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  6           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  7          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


### Closures

A closure is a function—let’s call it f—with an extended scope that encompasses
variables referenced in the body of f that are not global variables or local variables
of f. Such variables must come from the local scope of an outer function that
encompasses f.

It does not matter whether the function is anonymous or not; what matters is that it
can access nonglobal variables that are defined outside of its body.

In [14]:
# Example 9-7. average_oo.py: a class to calculate a running average

class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [15]:
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [16]:
# Example 9-8. average.py: a higher-order function to calculate a running average

def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

In [17]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(15))

10.0
10.5
12.0


In [21]:
# Example 9-10. Inspecting the function created by make_averager in Example 9-8

print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('new_value', 'total')
('series',)


In [18]:
# Example 9-11. Continuing from Example 9-9

print(avg.__code__.co_freevars)
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

('series',)
(<cell at 0x00000245A0102E30: list object at 0x00000245A1C770C0>,)
[10, 11, 15]


### The nonlocal Declaration

In [19]:
# Example 9-12. A broken higher-order function to calculate a running average without keeping all history

def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [20]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [21]:
# Example 9-13. Calculate a running average without keeping all history (fixed with the use of nonlocal)

def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [22]:
avg = make_averager()
avg(10)

10.0

In [1]:
# Example 9-14. clockdeco0.py: simple decorator to show the running time of functions

import time
#TODO 
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

In [2]:
# Example 9-15. Using the clock decorator

import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))
print()
print(factorial.__name__)

**************************************** Calling snooze(.123)
[0.13812970s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000720s] factorial(1) -> 1
[0.00002200s] factorial(2) -> 2
[0.00002840s] factorial(3) -> 6
[0.00003360s] factorial(4) -> 24
[0.00003950s] factorial(5) -> 120
[0.00004610s] factorial(6) -> 720
6! = 720

clocked


In [3]:
# Example 9-16. clockdeco.py: an improved clock decorator

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):  # support keyword arguments
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

In [4]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))
print()
print(factorial.__name__)

@clock
def add(a, b):
    return a+b

add(2,3)

**************************************** Calling factorial(6)
[0.00000060s] factorial(1) -> 1
[0.00002170s] factorial(2) -> 2
[0.00003190s] factorial(3) -> 6
[0.00004080s] factorial(4) -> 24
[0.00004970s] factorial(5) -> 120
[0.00005880s] factorial(6) -> 720
6! = 720

factorial
[0.00000070s] add(2, 3) -> 5


5

### Decorators in the Standard Library

In [19]:
# Momoization with functools.cache

# Example 9-17. The very costly recursive way to compute the nth number in the Fibonacci series

@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000070s] fibonacci(0) -> 0
[0.00000070s] fibonacci(1) -> 1
[0.00009590s] fibonacci(2) -> 1
[0.00000050s] fibonacci(1) -> 1
[0.00000070s] fibonacci(0) -> 0
[0.00000030s] fibonacci(1) -> 1
[0.00001570s] fibonacci(2) -> 1
[0.00003080s] fibonacci(3) -> 2
[0.00014390s] fibonacci(4) -> 3
[0.00000040s] fibonacci(1) -> 1
[0.00000030s] fibonacci(0) -> 0
[0.00000030s] fibonacci(1) -> 1
[0.00001530s] fibonacci(2) -> 1
[0.00003120s] fibonacci(3) -> 2
[0.00000040s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00001480s] fibonacci(2) -> 1
[0.00000050s] fibonacci(1) -> 1
[0.00000050s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00001450s] fibonacci(2) -> 1
[0.00002940s] fibonacci(3) -> 2
[0.00005820s] fibonacci(4) -> 3
[0.00010350s] fibonacci(5) -> 5
[0.00026260s] fibonacci(6) -> 8
8


In [22]:
# Example 9-18. Faster implementation using caching
import functools

@functools.cache
# @functools.lru_cache(maxsize=2**20, typed=True)
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

fibonacci(6)

[0.00000080s] fibonacci(0) -> 0
[0.00000150s] fibonacci(1) -> 1
[0.00050650s] fibonacci(2) -> 1
[0.00000130s] fibonacci(3) -> 2
[0.00053060s] fibonacci(4) -> 3
[0.00000080s] fibonacci(5) -> 5
[0.00055150s] fibonacci(6) -> 8


8

In [24]:
# Example 9-20. @singledispatch creates a custom @htmlize.register to bundle
# several functions into a generic function

from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'


@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

In [28]:
#Example 9-19. htmlize() generates HTML tailored to different object types
print(htmlize({1, 2, 3}))
print()
print(htmlize(abs))
print()
print(htmlize('Heimlich & Co.\n- a game'))
print()
print(htmlize(42))
print()
print(htmlize(['alpha', 66, {3, 2, 1}]))
print()
print(htmlize(True))
print()
print(htmlize(fractions.Fraction(2, 3)))
print()
print(htmlize(2/3))
print()
print(htmlize(decimal.Decimal('0.02380952')))

<pre>{1, 2, 3}</pre>

<pre>&lt;built-in function abs&gt;</pre>

<p>Heimlich &amp; Co.<br/>
- a game</p>

<pre>42 (0x2a)</pre>

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>

<pre>True</pre>

<pre>2/3</pre>

<pre>0.6666666666666666 (2/3)</pre>

<pre>0.02380952 (1/42)</pre>


### Parameterized Decorators

In [32]:
# Example 9-21. Abridged registration.py module from Example 9-2, repeated here for convenience

registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->', registry)
f1()


running register(<function f1 at 0x000002299633F7F0>)
running main()
registry -> [<function f1 at 0x000002299633F7F0>]
running f1()


In [33]:
# Example 9-22. To accept parameters, the new register decorator must be called as a function

registry = set()

def register(active=True):
    def decorate(func):
        print('running register'
            f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x00000229963F9120>)
running register(active=True)->decorate(<function f2 at 0x00000229963F8EE0>)


In [35]:
# Example 9-23. Using the registration_param module listed in Example 9-22

print(registry)

register()(f3)
print(registry)

register(active=False)(f2)
print(registry)

{<function f2 at 0x00000229963F8EE0>, <function f3 at 0x000002299633F7F0>}
running register(active=True)->decorate(<function f3 at 0x000002299633F7F0>)
{<function f2 at 0x00000229963F8EE0>, <function f3 at 0x000002299633F7F0>}
running register(active=False)->decorate(<function f2 at 0x00000229963F8EE0>)
{<function f3 at 0x000002299633F7F0>}


In [38]:
# Example 9-24. Module clockdeco_param.py: the parameterized clock decorator

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.13453820s] snooze(0.123) -> None
[0.12487040s] snooze(0.123) -> None
[0.12577670s] snooze(0.123) -> None


In [40]:
# Example 9-25. clockdeco_param_demo1.py

import time

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12677700001222547s
snooze: 0.12342639999405947s
snooze: 0.12494510000396986s


In [41]:
# Example 9-26. clockdeco_param_demo2.py

import time

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.137s
snooze(0.123) dt=0.124s
snooze(0.123) dt=0.126s


In [ ]:
# Example 9-27. Module clockdeco_cls.py: parameterized clock decorator implemented as class
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock:
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt

    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

In [16]:
from collections.abc import Iterable
import functools
# test
def unit_test_data_gen(func):
    @functools.wraps(func)
    def unit_test_data_gen(*args, **kwargs):
        for i, argument in enumerate(args):
            print(i, argument)
        for k, v in kwargs.items():
            print(k, v)
        result = func(*args, **kwargs)        
        if isinstance(result, Iterable):
            for i, ret in enumerate(result):
                print("Result: ", i, ret)
        else:
            print('Result:', result)
        return result
    return unit_test_data_gen

@unit_test_data_gen
def add(a, b):
    return a+b

add(2, 3)
add(a=3, b=4)

0 2
1 3
Result: 5
a 3
b 4
Result: 7


7